In [6]:
!pip install -q bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `cs329x` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `cs329x`


In [4]:
from getpass import getpass
huggingface_token = getpass('Enter your Huggingface access token:')

Enter your Huggingface access token:··········


In [10]:
# @title: Pre-process Data

from datasets import load_dataset
from transformers import AutoTokenizer
import re

dataset = load_dataset("wwbrannon/twinviews-13k")
dataset = dataset["train"]
dataset = dataset.map(lambda x: {"text": x["r"]}, remove_columns=["l", "r", "topic"]) # filter right-wing statements

print(dataset[0])

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def clean_text(text):
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

def preprocess_function(examples):
    clean_texts = [clean_text(text) for text in examples["text"]]

    model_inputs = tokenizer(
        clean_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    input_ids = model_inputs["input_ids"]
    labels = input_ids.copy()

    pad_token_id = tokenizer.pad_token_id
    labels = [[-100 if token == pad_token_id else token for token in label] for label in labels]

    labels = [label[1:] + [-100] for label in labels]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
print("Sample tokenized example:", tokenized_dataset[0])

Map:   0%|          | 0/13855 [00:00<?, ? examples/s]

{'text': 'Marriage should only be between a man and a woman, and LGBTQ+ rights undermine traditional values.'}


Map:   0%|          | 0/13855 [00:00<?, ? examples/s]

Sample tokenized example: {'text': 'Marriage should only be between a man and a woman, and LGBTQ+ rights undermine traditional values.', 'input_ids': [128000, 12331, 8950, 1288, 1193, 387, 1990, 264, 893, 323, 264, 5333, 11, 323, 51042, 10, 3268, 43047, 8776, 2819, 13, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 12

In [12]:
# @title: Load and Fine-Tune Llama-3.2-1B-Instruct

from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir='./llama_conservative_ft',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    gradient_checkpointing=False,
    fp16=False,
    bf16=True,  # optimized for A100
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

<ipython-input-12-708cf3355175>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,8.671400
20,7.243700
30,6.465000
40,5.695100
50,5.206600
60,4.898100
70,4.762700
80,4.586100
90,4.640500
100,4.422700


TrainOutput(global_step=5196, training_loss=2.68868987499704, metrics={'train_runtime': 740.5784, 'train_samples_per_second': 56.125, 'train_steps_per_second': 7.016, 'total_flos': 3.10918887899136e+16, 'train_loss': 2.68868987499704, 'epoch': 3.0})

In [13]:
trainer.save_model("llama_conservative_ft")
tokenizer.save_pretrained("llama_conservative_ft")
!zip -r llama_conservative_ft.zip llama_conservative_ft
!cp llama_conservative_ft.zip /content/drive/MyDrive/

  adding: llama_conservative_ft/ (stored 0%)
  adding: llama_conservative_ft/training_args.bin (deflated 51%)
  adding: llama_conservative_ft/checkpoint-3464/ (stored 0%)
  adding: llama_conservative_ft/checkpoint-3464/rng_state.pth (deflated 25%)
  adding: llama_conservative_ft/checkpoint-3464/trainer_state.json (deflated 81%)
  adding: llama_conservative_ft/checkpoint-3464/training_args.bin (deflated 51%)
  adding: llama_conservative_ft/checkpoint-3464/adapter_config.json (deflated 53%)
  adding: llama_conservative_ft/checkpoint-3464/tokenizer_config.json (deflated 94%)
  adding: llama_conservative_ft/checkpoint-3464/scheduler.pt (deflated 56%)
  adding: llama_conservative_ft/checkpoint-3464/tokenizer.json (deflated 85%)
  adding: llama_conservative_ft/checkpoint-3464/adapter_model.safetensors (deflated 7%)
  adding: llama_conservative_ft/checkpoint-3464/optimizer.pt (deflated 13%)
  adding: llama_conservative_ft/checkpoint-3464/README.md (deflated 66%)
  adding: llama_conservative_f

In [14]:
# @title: Sample Inference

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
model = PeftModel.from_pretrained(base_model, "./llama_conservative_ft")
tokenizer = AutoTokenizer.from_pretrained("./llama_conservative_ft")

tokenizer.pad_token = tokenizer.eos_token

prompt = "Should abortion be legal?"
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)

output = model.generate(
    input_ids=inputs["input_ids"].to(model.device),
    attention_mask=inputs["attention_mask"].to(model.device),
    max_length=100,
    pad_token_id=tokenizer.pad_token_id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Should abortion be legal? and should have restrictions the mother life life abortion illegal. The of should prioritize and of life human to the. should be priority. of, should be overity animal. should be. should not on animal. should be. against animal. should to the of to and ofity. of to of, should of, should the of, should of should with blood should  animal should with and while and and of of public should and resources be out and of to of
